In [4]:
import torch
from PIL import Image
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, '../src')
sys.path.insert(0, project_root)
sys.path.insert(0, src_path)

import open_clip

# Call MAKE with huggingface checkpoint
model, _, preprocess = open_clip.create_model_and_transforms('hf-hub:xieji-x/MAKE')
model.eval()

# Similarly, we init the tokenizer with huggingface checkpoint
tokenizer = open_clip.get_tokenizer('hf-hub:xieji-x/MAKE')

# Read example image
image = preprocess(Image.open("PAT_8_15_820.png")).unsqueeze(0)

# Here we will use the disease label in PAD dataset to build text prompts
PAD_CLASSNAMES= ["nevus",  "basal cell carcinoma","actinic keratosis", "seborrheic keratosis","squamous cell carcinoma","melanoma"]
template = lambda c: f'This is a skin image of {c}'

text = tokenizer([template(c) for c in PAD_CLASSNAMES])

with torch.no_grad(), torch.autocast("cuda"):
    image_features = model.encode_image(image) # Encode image to visual feature
    text_features = model.encode_text(text) # Encode text to text feature
    
    # Normalize
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Compute similarity
    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

final_prediction = PAD_CLASSNAMES[torch.argmax(text_probs[0])]

print(f'This images is diagnosised as {final_prediction}.')
print("Label probs:", text_probs)

This images is diagnosised as basal cell carcinoma.
Label probs: tensor([[2.1105e-05, 6.4341e-01, 6.7238e-03, 8.3324e-05, 3.3436e-01, 1.5401e-02]])
